# Assignment 2

## Roll number: 


### Instructions
 * Fill in the roll-number in the cell above.
 * Code must be submitted in Python in jupyter notebooks. We highly recommend using anaconda/miniconda distribution or at the minimum, virtual environments for this assignment.
 * All the code and result files should be uploaded in the github classroom.
 *  Most of the questions require you to **code your own functions** unless there is a need to call in the abilities of the mentioned libraries, such as Visualisation from Open3D. Make sure your code is modular since you will be reusing them for future assignments. All the functions related to transformation matrices, quaternions, and 3D projection are expected to be coded by you.
 *  All the representations are expected to be in a right-hand coordinate system.
<!--  * Answer to the descriptive questions should be answered in your own words. Copy-paste answers will lead to penalty. -->
 * You could split the Jupyter Notebook cells where TODO is written, but please try to avoid splitting/changing the structure of other cells.
 * All the visualization should be done inside the notebook unless specified otherwise.
 * Plagiarism will lead to heavy penalty.
 * Commit the notebooks in the repo and any other results files under the result folder in the GitHub Classroom repo. 
 * This is a group assignment. Discussions are encouraged but any sharing of code among different teams will be penalized. 

In [2]:
import os
import time
import numpy as np
import open3d as o3d
import math
import cv2
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## SECTION 1: ICP with SVD

### 1.1 Perform Procrustes alignmenton two point clouds with (given) known correspondences. (5 Points)

Let X be your point cloud observed from the initial pose P1. You then transform it to a new pose P2. Now you wish to apply ICP to recover transformation between (X & P1) and (X & P2).

Use toothless.ply point cloud and perform the alignment between the two point clouds using procrustes alignment. Your task is to write a function that takes two point clouds as input wherein the corresponding points between the two point clouds are located at the same index and returns the transformation matrix between them. Compute the alignment error after aligning the two point clouds.

<b>Use root mean squared error (RSME) as the alignment error metric.</b>

Make sure your code is modular as we will use this function in the next sub-part.

We will again use our own getTransform function to generate the 4x4 Transformation matrix to transform the pointcloud to P2, so make sure your code works for any general Transformation matrix

In [16]:
def alpha_rot(alpha):
    return np.array([
        [   1.0,    0.0,                0.0             ],
        [   0.0,    math.cos(alpha),   -math.sin(alpha) ],
        [   0.0,    math.sin(alpha),    math.cos(alpha) ]
        ],dtype=np.float32)

def beta_rot(beta):
    return np.array([
        [   math.cos(beta),     0.0,    math.sin(beta)  ],
        [   0.0,                1.0,    0.0             ],
        [  -math.sin(beta),     0.0,    math.cos(beta)  ]
        ],dtype=np.float32)

def gamma_rot(gamma):
    return np.array([
        [   math.cos(gamma),   -math.sin(gamma),    0.0 ],
        [   math.sin(gamma),    math.cos(gamma),    0.0 ],
        [   0.0,                0.0,                1.0 ]
        ],dtype=np.float32)\
        
def euler2rm(alpha, beta, gamma):
    return np.matmul(alpha_rot(alpha),np.matmul(beta_rot(beta),gamma_rot(gamma)))

In [17]:
def centre_of_mass_func(points):
    return np.mean(points, axis=0)

def error(target, predicted, transform_matrix):
    return 

In [18]:
def getTransform():
    # Random rotation matrices for X, Y, and Z axes
    R = euler2rm(np.random.uniform(0, 2 * np.pi),
                 np.random.uniform(0, 2 * np.pi),
                 np.random.uniform(0, 2 * np.pi))
    
    # Random translation matrix
    T = np.array([
        [np.random.uniform(-500, 500)],
        [np.random.uniform(-500, 500)],
        [np.random.uniform(-500, 500)]
    ])
    
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = R
    transformation_matrix[:3, 3] = T.flatten()
    
    return transformation_matrix

In [37]:
def icp_known_correspondences(T, pcd):
    #### YOUR CODE HERE ####
    # P1 point cloud (initial pose)
    P1 = pcd

    # P2 point cloud (Transform pose)
    print(T)
    P2 = pcd.transform(T)

    # Numpy array convertion
    P1_np = np.asarray(P1.points)
    P2_np = np.asarray(P2.points)

    # Calculate centre of Mass for P1, P2
    centre_mass_P1 = centre_of_mass_func(P1_np)
    centre_mass_P2 = centre_of_mass_func(P2_np)

    # Normalisation of P1 and P2
    P1_norm = P1_np - centre_mass_P1
    P2_norm = P2_np - centre_mass_P2

    # Cross-Covariance Matrix
    W = P1_norm.T @ P2_norm

    # SVD
    U,D,Vt = np.linalg.svd(W)

    # Rotational Matrix and Translation Matrix
    R = U @ Vt
    t = centre_mass_P1 - (R @ centre_mass_P2)
    t = np.expand_dims(t, axis=-1)

    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3:] = t

    return T 

# CODE 
T = getTransform()
point_cloud_path = "data/toothless.ply"
pcd = o3d.io.read_point_cloud(point_cloud_path)
voxel_size = 5.0
downsampled_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
transform_matrix = icp_known_correspondences(T, downsampled_pcd)
alignment_error = np.linalg.norm(transform_matrix - T)
alignment_error

[[ 1.28572121e-01  2.09519267e-01  9.69314635e-01  1.78174213e+02]
 [-1.17578976e-01 -9.67312276e-01  2.24682376e-01  3.74621347e+02]
 [ 9.84705269e-01 -1.42858908e-01 -9.97343212e-02 -3.11425021e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


518.729818034171

### 1.2 Implement ICP algorithm with unknown correspondences. (5 Points)

Your task is to write a function that implements ICP and takes two point clouds as input wherein the correspondances are unknown. Visualize the pointclouds and plot their individual coordinate frames as you perform ICP over them. Compute the alignment error in each iteration. 

Refer to Shubodh's notes to compute correspondences: https://saishubodh.notion.site/Mobile-Robotics-Navigating-from-Theory-to-Application-0b65a9c20edd4081978f4ffad917febb?p=a25686ce1a11409d838d47bcac43ab4b&pm=s#bb9aaf2e316b4db3b399df1742f0444c


In [ ]:
P = getTransform()
Q = getTransform()

def icp_unknown_correspondences(P, Q):
    #### YOUR CODE HERE ####
    pass

## SECTION 2: ICP with Lie Groups

### 2.1 Predict the Transformation matrix between 2 point clouds with known correspondences (15 Points)
Perform the same task as 1.1 using Lie Group Optimization from scratch to predict the transformation between the 2 point clouds.

Refer: https://saishubodh.notion.site/Mobile-Robotics-Navigating-from-Theory-to-Application-0b65a9c20edd4081978f4ffad917febb?p=ee55fe5689794693910ab7861bef067b&pm=s#7b82d84766a84b63b91d859579e4886b


In [ ]:
T = getTransform()
def icp_with_lie(T):
    #### YOUR CODE HERE ####
    pass

## SECTION 3: Pose Graph Optimization with G2O
### Objective (5 Points)
A robot is travelling in a oval trajectory. It is equipped with wheel odometry for odometry information and RGBD sensors for loop closure information. Due to noise in wheel odometry it generates a noisy estimate of the trajectory. Our task is to use loop closure pairs to correct the drift.

We pose this problem as a pose graph optimization problem. In our graph, poses are the vertices and constraints are the edges. 

References:

1.) Class notes: https://saishubodh.notion.site/G2O-Edge-Types-d9f9ff63c77c4ceeb84b1e49085004e3

2.) Cyrill Stachniss lecture: https://www.youtube.com/watch?v=uHbRKvD8TWg 

### Given: 
In practical scenarios, we'd obtain the following from our sensors after some post-processing:

1. Initial position
2. Odometry Contraints/Edges: This "edge" information tells us relative transformation between two nodes. These two nodes are consecutive in the case of Odometry but not in the case of Loop Closure (next point).
3. Loop Closure Contraints/Edges: Remember that while optimizing, you have another kind of "anchor" edge as you've seen in 1. solved example.

You have been given a text file named `edges.txt` (in `data/`) which has all the above 3 and it follows G2O's format (as explained in class, [link here](https://saishubodh.notion.site/G2O-Edge-Types-d9f9ff63c77c4ceeb84b1e49085004e3) ). The ground truth is `gt.txt`.

Install g2o as mentioned in `g2o.ipynb` and optimise `edges.txt`

In [ ]:
### YOUR CODE HERE ###

### Evo (10 Points)
We need a measure of how good the trajectory is. The error/loss used earlier doesn't tell us much about how the trajectory differs from the ground truth. Here, we try to do just this - compute error metrics. Rather than computing these from scratch, we will just Evo - https://github.com/MichaelGrupp/evo/.

Look at the absolute pose error (APE) and relative pose error (RPE). What do they capture and how are they calculated (descriptive answer)? How do these metrics differ in methodology? Can we determine if the error is more along the x/y axis?

Answer the above questions and report errors for the obtained trajectory.

In [ ]:
### YOUR CODE HERE ###